In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MABarbadillo/
%cd 'GWP/GWP files'
import os

Mounted at /content/drive
/content/drive/MyDrive/MABarbadillo
/content/drive/MyDrive/MABarbadillo/GWP/GWP files


In [13]:
##Install rosetta

!pip install rosetta-soil
from rosetta import rosetta, SoilData

In [45]:
import numpy as np
import pandas as pd

# Read the CSV file, skipping the first row and using the second row as header
df = pd.read_csv('/content/drive/MyDrive/MABarbadillo/GWP/GWP files/MesoSoilv1_3.csv', header=0, skiprows=1)

# Extract the data from the specified columns
data = df.iloc[1:, [2, 3, 4, 5, 6, 7]].values.tolist()

# Call the function to get hydraulic parameters

mean_1, stdev_1, codes_1 = rosetta(1, SoilData.from_array(data))
mean_3, stdev_3, codes_3 = rosetta(3, SoilData.from_array(data))

# Print results
print("Mean:")
print(mean_1)
print(mean_3)
#print("Standard Deviation:")
#print(stdev_3)
#print(stdev_1)

Mean:
[[ 0.01960586  0.4322467  -1.31364824  0.13058906  2.28911681]
 [        nan         nan         nan         nan         nan]
 [ 0.03641117  0.42522559 -1.46516696  0.12924389  1.91250989]
 ...
 [ 0.10019199  0.58924057 -1.80231901  0.14556029  2.11345023]
 [ 0.07225008  0.47702418 -2.26716735  0.15989478  1.20833645]
 [ 0.08681489  0.39276817 -1.81065384  0.09492901  0.31889299]]
[[ 0.03405871  0.4100014  -1.5719425   0.14339267  2.34259722]
 [        nan         nan         nan         nan         nan]
 [ 0.05002338  0.41191086 -1.69384262  0.13455548  1.95340167]
 ...
 [ 0.11963068  0.56802636 -2.15476872  0.134639    2.04613654]
 [ 0.10019103  0.46240308 -2.31179518  0.13745604  1.25378786]
 [ 0.13377872  0.38796566 -2.11279715  0.08938117  0.25665011]]
